# Preparing the dataset

In [ ]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

-rw-r--r-- 1 root root 70 Nov 17 12:52 kaggle.json


In [ ]:
!kaggle datasets download -d Cornell-University/arxiv

100% 1.09G/1.09G [00:29<00:00, 43.3MB/s]
100% 1.09G/1.09G [00:29<00:00, 40.2MB/s]


In [ ]:
!unzip arxiv.zip

Archive:  arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Introduction

#### In this notebook, we will be using multiple Transformer model to perform title generation from abstract. We will also be generating a score telling how good a title is:-

### Libraries 📚⬇

In [ ]:
%%time
!pip install -q tqdm
!pip install tokenizers
!pip install transformers
!pip install simpletransformers
!pip install numba 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 250 ms, sys: 65 ms, total: 315 ms
Wall time: 30 s


In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.12.1+cu113', '4.24.0', '0.13.2')

### Read JSON Data 📝

In [ ]:
data_file = 'arxiv-metadata-oai-snapshot.json'

""" Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly"""

def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

#### Let's print a sample paper metadata

In [ ]:
metadata = get_metadata()
for paper in metadata:
    for k, v in json.loads(paper).items():
        print(f'{k}: {v} \n')
    break

id: 0704.0001 

submitter: Pavel Nadolsky 

authors: C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan 

title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies 

comments: 37 pages, 15 figures; published version 

journal-ref: Phys.Rev.D76:013009,2007 

doi: 10.1103/PhysRevD.76.013009 

report-no: ANL-HEP-PR-07-12 

categories: hep-ph 

license: None 

abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
m

In [ ]:
category_map = {'astro-ph': 'Astrophysics',
                'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
                'astro-ph.EP': 'Earth and Planetary Astrophysics',
                'astro-ph.GA': 'Astrophysics of Galaxies',
                'astro-ph.HE': 'High Energy Astrophysical Phenomena',
                'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
                'astro-ph.SR': 'Solar and Stellar Astrophysics',
                'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
                'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
                'cond-mat.mtrl-sci': 'Materials Science',
                'cond-mat.other': 'Other Condensed Matter',
                'cond-mat.quant-gas': 'Quantum Gases',
                'cond-mat.soft': 'Soft Condensed Matter',
                'cond-mat.stat-mech': 'Statistical Mechanics',
                'cond-mat.str-el': 'Strongly Correlated Electrons',
                'cond-mat.supr-con': 'Superconductivity',
                'cs.AI': 'Artificial Intelligence',
                'cs.AR': 'Hardware Architecture',
                'cs.CC': 'Computational Complexity',
                'cs.CE': 'Computational Engineering, Finance, and Science',
                'cs.CG': 'Computational Geometry',
                'cs.CL': 'Computation and Language',
                'cs.CR': 'Cryptography and Security',
                'cs.CV': 'Computer Vision and Pattern Recognition',
                'cs.CY': 'Computers and Society',
                'cs.DB': 'Databases',
                'cs.DC': 'Distributed, Parallel, and Cluster Computing',
                'cs.DL': 'Digital Libraries',
                'cs.DM': 'Discrete Mathematics',
                'cs.DS': 'Data Structures and Algorithms',
                'cs.ET': 'Emerging Technologies',
                'cs.FL': 'Formal Languages and Automata Theory',
                'cs.GL': 'General Literature',
                'cs.GR': 'Graphics',
                'cs.GT': 'Computer Science and Game Theory',
                'cs.HC': 'Human-Computer Interaction',
                'cs.IR': 'Information Retrieval',
                'cs.IT': 'Information Theory',
                'cs.LG': 'Machine Learning',
                'cs.LO': 'Logic in Computer Science',
                'cs.MA': 'Multiagent Systems',
                'cs.MM': 'Multimedia',
                'cs.MS': 'Mathematical Software',
                'cs.NA': 'Numerical Analysis',
                'cs.NE': 'Neural and Evolutionary Computing',
                'cs.NI': 'Networking and Internet Architecture',
                'cs.OH': 'Other Computer Science',
                'cs.OS': 'Operating Systems',
                'cs.PF': 'Performance',
                'cs.PL': 'Programming Languages',
                'cs.RO': 'Robotics',
                'cs.SC': 'Symbolic Computation',
                'cs.SD': 'Sound',
                'cs.SE': 'Software Engineering',
                'cs.SI': 'Social and Information Networks',
                'cs.SY': 'Systems and Control',
                'econ.EM': 'Econometrics',
                'eess.AS': 'Audio and Speech Processing',
                'eess.IV': 'Image and Video Processing',
                'eess.SP': 'Signal Processing',
                'gr-qc': 'General Relativity and Quantum Cosmology',
                'hep-ex': 'High Energy Physics - Experiment',
                'hep-lat': 'High Energy Physics - Lattice',
                'hep-ph': 'High Energy Physics - Phenomenology',
                'hep-th': 'High Energy Physics - Theory',
                'math.AC': 'Commutative Algebra',
                'math.AG': 'Algebraic Geometry',
                'math.AP': 'Analysis of PDEs',
                'math.AT': 'Algebraic Topology',
                'math.CA': 'Classical Analysis and ODEs',
                'math.CO': 'Combinatorics',
                'math.CT': 'Category Theory',
                'math.CV': 'Complex Variables',
                'math.DG': 'Differential Geometry',
                'math.DS': 'Dynamical Systems',
                'math.FA': 'Functional Analysis',
                'math.GM': 'General Mathematics',
                'math.GN': 'General Topology',
                'math.GR': 'Group Theory',
                'math.GT': 'Geometric Topology',
                'math.HO': 'History and Overview',
                'math.IT': 'Information Theory',
                'math.KT': 'K-Theory and Homology',
                'math.LO': 'Logic',
                'math.MG': 'Metric Geometry',
                'math.MP': 'Mathematical Physics',
                'math.NA': 'Numerical Analysis',
                'math.NT': 'Number Theory',
                'math.OA': 'Operator Algebras',
                'math.OC': 'Optimization and Control',
                'math.PR': 'Probability',
                'math.QA': 'Quantum Algebra',
                'math.RA': 'Rings and Algebras',
                'math.RT': 'Representation Theory',
                'math.SG': 'Symplectic Geometry',
                'math.SP': 'Spectral Theory',
                'math.ST': 'Statistics Theory',
                'math-ph': 'Mathematical Physics',
                'nlin.AO': 'Adaptation and Self-Organizing Systems',
                'nlin.CD': 'Chaotic Dynamics',
                'nlin.CG': 'Cellular Automata and Lattice Gases',
                'nlin.PS': 'Pattern Formation and Solitons',
                'nlin.SI': 'Exactly Solvable and Integrable Systems',
                'nucl-ex': 'Nuclear Experiment',
                'nucl-th': 'Nuclear Theory',
                'physics.acc-ph': 'Accelerator Physics',
                'physics.ao-ph': 'Atmospheric and Oceanic Physics',
                'physics.app-ph': 'Applied Physics',
                'physics.atm-clus': 'Atomic and Molecular Clusters',
                'physics.atom-ph': 'Atomic Physics',
                'physics.bio-ph': 'Biological Physics',
                'physics.chem-ph': 'Chemical Physics',
                'physics.class-ph': 'Classical Physics',
                'physics.comp-ph': 'Computational Physics',
                'physics.data-an': 'Data Analysis, Statistics and Probability',
                'physics.ed-ph': 'Physics Education',
                'physics.flu-dyn': 'Fluid Dynamics',
                'physics.gen-ph': 'General Physics',
                'physics.geo-ph': 'Geophysics',
                'physics.hist-ph': 'History and Philosophy of Physics',
                'physics.ins-det': 'Instrumentation and Detectors',
                'physics.med-ph': 'Medical Physics',
                'physics.optics': 'Optics',
                'physics.plasm-ph': 'Plasma Physics',
                'physics.pop-ph': 'Popular Physics',
                'physics.soc-ph': 'Physics and Society',
                'physics.space-ph': 'Space Physics',
                'q-bio.BM': 'Biomolecules',
                'q-bio.CB': 'Cell Behavior',
                'q-bio.GN': 'Genomics',
                'q-bio.MN': 'Molecular Networks',
                'q-bio.NC': 'Neurons and Cognition',
                'q-bio.OT': 'Other Quantitative Biology',
                'q-bio.PE': 'Populations and Evolution',
                'q-bio.QM': 'Quantitative Methods',
                'q-bio.SC': 'Subcellular Processes',
                'q-bio.TO': 'Tissues and Organs',
                'q-fin.CP': 'Computational Finance',
                'q-fin.EC': 'Economics',
                'q-fin.GN': 'General Finance',
                'q-fin.MF': 'Mathematical Finance',
                'q-fin.PM': 'Portfolio Management',
                'q-fin.PR': 'Pricing of Securities',
                'q-fin.RM': 'Risk Management',
                'q-fin.ST': 'Statistical Finance',
                'q-fin.TR': 'Trading and Market Microstructure',
                'quant-ph': 'Quantum Physics',
                'stat.AP': 'Applications',
                'stat.CO': 'Computation',
                'stat.ME': 'Methodology',
                'stat.ML': 'Machine Learning',
                'stat.OT': 'Other Statistics',
                'stat.TH': 'Statistics Theory'}

# #1. Abstract Summarization
###    Title Prediction from Abstract using **seq2seq** models

#### For this task, let's choose all paper categories published in the last decade

In [ ]:
titles = []
abstracts = []

# Consider all categories in the `category_map` to be used during training and prediction
paper_categories = np.array(list(category_map.keys())).flatten()

# # Consider specific paper categories to be used during training and prediction
# paper_categories = ["cs.AI", # Artificial Intelligence
#                     "cs.CV", # Computer Vision and Pattern Recognition
#                     "cs.LG"] # Machine Learning

metadata = get_metadata()
for paper in tqdm(metadata):
    paper_dict = json.loads(paper)
    category = paper_dict.get('categories')
    try:
        year = int(paper_dict.get('journal-ref')[-4:])
        if category in paper_categories and 2019<year<2021:
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract').replace("\n",""))
    except:
        pass 

len(titles), len(abstracts)

0it [00:00, ?it/s]

(2329, 2329)

### #1.2 Data Pre-processing ⚙️

In [ ]:
papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
})

papers = papers[['abstract', 'title']]
papers.columns = ['input_text', 'target_text']
papers = papers.dropna()

del titles, abstracts
papers

,input_text,target_text
0,We consider a nonequilibrium process on a ti...,Nonequilibrium phase transition in a spreading...
1,"We consider 1-dimensional, unimodular Pisot ...",MLD Relations of Pisot Substitution Tilings
2,One of the main proposed tools to transfer i...,Long-range interactions and information transf...
3,We investigate by Monte Carlo simulations th...,Anomalous zipping dynamics and forced polymer ...
4,The effects of the ion Larmor radius on magn...,Gyrofluid simulations of collisionless reconne...
...,...,...
2324,Robotic hands with soft surfaces can perform...,Sensing and Control of Friction Mode for Conta...
2325,"In this paper, we introduce some fundamental...",Stochastic derivatives for fractional diffusions
2326,"Geometric models and Teichm\""uller structure...","Function Models for Teichm\""uller Spaces and D..."
2327,We consider an integrable scalar partial dif...,An integrable hierarchy with a perturbed Henon...


### #1.3 Model Training & Evaluation 🚞

In [ ]:
eval_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(eval_df.index)

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
    "evaluate_generated_text": True,
}


# Roberta (Seq2Seq) Model

In [ ]:
from simpletransformers.seq2seq import Seq2SeqModel

roberta = Seq2SeqModel(
    "roberta",
    "roberta-base",
    "bert-base-cased",
)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.6.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.6.crossattention.output.dense.weight', 'bert.encoder.layer.7.crossattention.self.value.weight', 'bert.encoder.layer.6.crossattention.output.LayerNorm.bias', 'be

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
%%time
roberta.use_multiprocessing = False
# Train the model
roberta.train_model(train_df)

# Evaluate the model
result = roberta.eval_model(eval_df)
print(result)

  0%|          | 0/2096 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 5.460941927897996e-05}
CPU times: user 1min 27s, sys: 11.9 s, total: 1min 39s
Wall time: 2min 1s


In [ ]:
!cp -r outputs "/content/drive/MyDrive/NLP Assignment/Roberta"

In [ ]:
!rm -rf outputs

# BART (Seq2Seq) Model

In [ ]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel
# Create a Bart-base model
BART = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

CPU times: user 2.72 s, sys: 248 ms, total: 2.97 s
Wall time: 4.85 s


In [ ]:
%%time

# Train the model
BART.train_model(train_df)

# Evaluate the model
result = BART.eval_model(eval_df)
print(result)

  0%|          | 0/2096 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 0.08908990360796451}
CPU times: user 6min 52s, sys: 48 s, total: 7min 40s
Wall time: 8min 26s


In [ ]:
!cp -r outputs "/content/drive/MyDrive/NLP Assignment/BART/"

In [ ]:
!rm -rf outputs

# T5 Model

In [ ]:
p2 = papers.copy()
# Adding <prefix> columns
p2['prefix'] = "summarize"
# splitting the data into training and test dataset
eval_p2 = p2.sample(frac=0.2, random_state=101)
train_p2 = p2.drop(eval_df.index)

In [ ]:
from simpletransformers.t5 import T5Model
T5 = T5Model(
    "t5",
    "t5-base",
    args=model_args,
)

In [ ]:
T5.args.use_multiprocessing = False
T5.train_model(train_p2)
results = T5.eval_model(eval_p2)
print(results)

  0%|          | 0/2096 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/59 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/466 [00:00<?, ?it/s]

{'eval_loss': 1.7506536406985784}


In [ ]:
!cp -r outputs "/content/drive/MyDrive/NLP Assignment/T5"

# Marian(Seq2Seq) Model

In [ ]:
from simpletransformers.seq2seq import Seq2SeqModel

Marian = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="Helsinki-NLP/opus-mt-en-de",
    args=model_args
)


Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/768k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

In [ ]:
%%time

# Train the model
Marian.train_model(train_df)
# Evaluate the model
result = Marian.eval_model(eval_df)
print(result)

  0%|          | 0/2096 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 0.13374413773417473}
CPU times: user 4min 45s, sys: 23.7 s, total: 5min 8s
Wall time: 5min 18s


In [ ]:
!cp -r outputs "/content/drive/MyDrive/NLP Assignment/Marian"

In [ ]:
!tensorboard dev upload --logdir ./runs \
  --name "Simple experiment with Seq2SeqModels" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./runs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=FvIJe2HOH1xrkSutx8N22ZzSe1j2Mz&prompt=consent&access_type=offline
Enter 

In [ ]:
from IPython.display import IFrame
IFrame(
    src="https://tensorboard.dev/experiment/6HPf29eYQwaBtquBfK9ZFg/",
    width="100%", height="800px")

### #1.4 Prediction 🔮

#### Loading models back in

In [ ]:
roberta = Seq2SeqModel(
    "roberta",
    encoder_decoder_name="Roberta",
    args=model_args,
)

Marian = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="Marian",
    args=model_args
)
T5 = T5Model(
    "t5",
    "T5",
    args=model_args,
)
# Create a Bart-base model
BART = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="BART",
                    args=model_args)


In [ ]:
# We will do this on random 25 abstracts
test_abstracts = []
test_titles = []
roberta_preds = []
bart_preds = []
t5_preds = []
marian_preds = []
for _ in range(25):

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # saving for further use
    test_abstracts.append(abstract)
    test_titles.append(true_title)

    # Predict with roberts
    roberta_predicted_title = roberta.predict([abstract])[0]
    BART_predicted_title = BART.predict([abstract])[0]
    T5_predicted_title = T5.predict([abstract])[0]
    marian_predicted_title = Marian.predict([abstract])[0]

    # saving for further use
    roberta_preds.append(roberta_predicted_title)
    bart_preds.append(BART_predicted_title)
    t5_preds.append(T5_predicted_title)
    marian_preds.append(marian_predicted_title)

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title (Roberta): {roberta_predicted_title}\n')
    print(f'Predicted Title (BART): {BART_predicted_title}\n')
    print(f'Predicted Title (T5): {T5_predicted_title}\n')
    print(f'Predicted Title (Marian): {marian_predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: Collecting data with a mobile phone: Studies of mechanical laws such as
  energy and momentum conservation

Predicted Title (Roberta): 

Predicted Title (BART): Video Analysis of Collision of a Cue Ball and an Eight Ball

Predicted Title (T5): VideoExtractor: VideoExtractor: VideoExtractor for Discrete Analysis

Predicted Title (Marian): A Video of the Cue Ball Study

Abstract:   We use videos taken with a mobile phone to study conservation of energy,conservation of momentum, and the work-energy theorem by analyzing thecollision of a cue ball and the eight ball. A video of the full time sequence,starting from before the cue ball is struck until well after the collision, isrecorded with a mobile phone. The video is imported into Origin (free toteachers and students taking a class) [OriginLab Corporation, Origin, Version2020 (OriginLab Corporation, Northampton, MA, USA, 2020)], where thecoordinates of the balls are digitized frame-by-frame using the free VideoExtractor app. F

### Let's add an abstract as an input and see what our models are predicting

In [ ]:
abstract = input("Please enter an Abstract- : ")
print("Prediction of BART \n" + BART.predict([abstract])[0])
print("Prediction of T5 \n" + T5.predict([abstract])[0])
print("Prediction of Marian \n" + Marian.predict([abstract])[0])


Please enter an Abstract- : To achieve control objectives for extremely large-scale complex networksusing standard methods is essentially intractable. In this work a theory of theapproximate control of complex network systems is proposed and developed by theuse of graphon theory and the theory of infinite dimensional systems. First,graphon dynamical system models are formulated in an appropriate infinitedimensional space in order to represent arbitrary-size networks of lineardynamical systems, and to define the convergence of sequences of networksystems with limits in the space. Exact controllability and approximatecontrollability of graphon dynamical systems are then investigated. Second, theminimum energy state-to-state control problem and the linear quadraticregulator problem for systems on complex networks are considered. The controlproblem for graphon limit systems is solved in each case and approximations aredefined which yield control laws for the original control problems.Furth

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Prediction of BART 
Approximate control of complex networks using graphon theory and the theory of infinite dimensional


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Prediction of T5 
Controllability and Approximation of Complex Network Systems using graphon Dynamical Systems


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Prediction of Marian 
Lineary Control for Lineary System with Limits


# Novelty
- We will compare titles generated by all models and find the best one.
- For this, we plan to use vector similarity metric. 
- What we do is to vectorize the actual title, and all titles generated by all models.
- We then compare these vectors/embeddings using vecor similarity and find the one most similar to the actual title.
- Based on the vector similarity metric, we can say which models performs best.
- We can also plot these embeddings and find out which models predict closer text and why. 

In [ ]:
# Creating the vectorizer
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.config.model_args import ModelArgs

model_args = ModelArgs(max_seq_length=100)

gpt_vectorizer = RepresentationModel(
    "gpt2", "gpt2",
    args=model_args
)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2ForTextRepresentation: ['h.9.ln_1.weight', 'h.7.ln_2.weight', 'h.6.mlp.c_fc.weight', 'h.3.mlp.c_proj.weight', 'h.2.mlp.c_fc.bias', 'h.7.ln_1.weight', 'h.5.mlp.c_proj.weight', 'h.7.ln_1.bias', 'h.0.attn.c_attn.weight', 'h.7.attn.bias', 'h.1.mlp.c_proj.bias', 'wpe.weight', 'h.2.ln_2.weight', 'h.0.mlp.c_proj.weight', 'h.6.attn.c_attn.weight', 'h.11.mlp.c_fc.weight', 'h.6.attn.bias', 'h.9.mlp.c_fc.weight', 'h.8.ln_1.bias', 'h.11.ln_1.bias', 'h.0.ln_1.weight', 'h.10.mlp.c_proj.bias', 'h.4.attn.c_proj.weight', 'ln_f.bias', 'h.6.mlp.c_proj.bias', 'h.10.attn.c_proj.weight', 'h.0.attn.c_attn.bias', 'h.10.mlp.c_fc.bias', 'h.10.mlp.c_proj.weight', 'h.9.attn.c_attn.weight', 'h.4.mlp.c_proj.bias', 'h.5.ln_1.weight', 'ln_f.weight', 'h.0.attn.c_proj.weight', 'h.2.attn.c_proj.weight', 'h.9.attn.bias', 'h.4.mlp.c_proj.weight', 'h.8.attn.c_proj.bias', 'h.0.attn.bias', 'h.10.mlp.c_fc.weight', 'h.3.attn.c_attn.weight', 'h.0.m

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
import numpy as np
import pandas as pd

for actual_title, roberta_title, bart_title, t5_title, marian_title in zip(test_titles,roberta_preds,bart_preds,t5_preds,marian_preds):
# for actual_title, roberta_title, bart_title,marian_title in zip(test_titles,roberta_preds,bart_preds,marian_preds):
  titles = [actual_title, roberta_title, bart_title, t5_title, marian_title]
  embeddings = gpt_vectorizer.encode_sentences(titles, combine_strategy="mean")

  base_vec = embeddings[0]

  rob = embeddings[1]
  bart = embeddings[2]
  t5 = embeddings[3]
  marian = embeddings[4]

  # compare the vectors
  roberta_score = np.dot(base_vec, rob)
  bart_score = np.dot(base_vec, bart)
  t5_score = np.dot(base_vec, t5)
  marian_score = np.dot(base_vec, marian)

  p = pd.DataFrame(columns=['Model','Similarity with actual'])
  p['Model'] = ['Roberta','BART','T5','Marian']
  p['Similarity with actual'] = [roberta_score,bart_score, t5_score, marian_score]

  print(p)
  print("The best model is:", p.iloc[p['Similarity with actual'].idxmax()])

  # p = pd.DataFrame(columns=['Model','Similarity with actual'])
  # p['Model'] = ['Roberta','BART','Marian']
  # p['Similarity with actual'] = [roberta_score,bart_score, marian_score]

  # print(p)
  # print("The best model is:", p.iloc[p['Similarity with actual'].idxmax()])
  print("-------------------------------------------------------------------")

     Model  Similarity with actual
0  Roberta              247.336548
1     BART              397.997162
2       T5              377.866394
3   Marian              388.262054
The best model is: Model                           BART
Similarity with actual    397.997162
Name: 1, dtype: object
-------------------------------------------------------------------
     Model  Similarity with actual
0  Roberta              247.184479
1     BART              396.681580
2       T5              364.009094
3   Marian              377.739075
The best model is: Model                          BART
Similarity with actual    396.68158
Name: 1, dtype: object
-------------------------------------------------------------------
     Model  Similarity with actual
0  Roberta              277.152954
1     BART              381.723724
2       T5              355.901917
3   Marian              327.973999
The best model is: Model                           BART
Similarity with actual    381.723724
Name: 1, dtype: 

### Let's take an abstract and see what our networks are predicting OR Let's see how good your title is:

In [ ]:
abstract = input("Enter Your Abstract")
actual_title = input("Enter your title")
titles = [actual_title, BART.predict([abstract])[0], T5.predict([abstract])[0], Marian.predict([abstract])[0]]
embeddings = gpt_vectorizer.encode_sentences(titles, combine_strategy="mean")

base_vec = embeddings[0]
bart = embeddings[1]
t5 = embeddings[2]
marian = embeddings[3]

# compare the vectors
best_score = np.dot(base_vec, base_vec)
bart_score = np.dot(base_vec, bart)
t5_score = np.dot(base_vec, t5)
marian_score = np.dot(base_vec, marian)

print("How good our BART title is:", (bart_score/best_score)*100)
print("How good our t5 title is:", (t5_score/best_score)*100)
print("How good our Marian title is:", (marian_score/best_score)*100)

Enter Your Abstract  The paper deals with a Bolza optimal control problem for a dynamical systemwhich motion is described by a delay differential equation under an initialcondition defined by a piecewise continuous function. For the value functionalin this problem, the Cauchy problem for the Hamilton-Jacobi-Bellman equationwith coinvariant derivatives is considered. Minimax and viscosity solutions ofthis problem are studied. It is proved that both of these solutions exist, areunique and coincide with the value functional.
Enter your titleMinimax and Viscosity Solutions of Hamilton-Jacobi-Bellman Equations for   Time-Delay Systems


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

How good our BART title is: 56.85131549835205
How good our t5 title is: 61.209750175476074
How good our Marian title is: 52.422136068344116
